In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
df_features = pd.read_csv("./4_combine_description_and_features.csv")

df_id = pd.read_csv("./2_id_with_link.csv")

# Loc null description & full tags

In [3]:
blank_id = []
crawl_id = []

for i in range(len(df_features)):
    tags = df_features.at[i, "story_tags_list"]
    description = df_features.at[i, "Description"]
    id_story = df_features.at[i, "ID"]
    
    if pd.isnull(description):
        blank_id.append(id_story)
        continue
    
    if type(tags) == type(np.nan) or type(tags) == type(math.nan):
        crawl_id.append(id_story)
        continue
    
    if len(tags) < 5:
        crawl_id.append(id_story)
        continue

In [4]:
print(blank_id)
print(crawl_id)

[307, 320, 401, 414, 437, 477, 524, 697, 705, 754, 981, 1574, 1858, 1945, 2067, 2190, 2210, 2707, 2768, 2800, 2827, 2883, 2885, 2951]
[112, 114, 554, 612, 795, 817, 952, 986, 994, 1157, 1171, 1208, 1230, 1237, 1247, 1260, 1276, 1300, 1369, 1488, 1550, 1586, 1601, 1658, 1794, 1876, 1886, 1894, 1897, 1967, 1968, 1979, 1987, 1996, 2091, 2096, 2243, 2244, 2254, 2267, 2283, 2325, 2341, 2367, 2381, 2414, 2433, 2489, 2505, 2565, 2637, 2660, 2669, 2703, 2708, 2724, 2748, 2760, 2766, 2767, 2788, 2793, 2794, 2939]


In [5]:
df_loc_description = df_features.loc[~df_features["ID"].isin(blank_id)]

In [6]:
df_to_crawl_tags = df_loc_description.loc[df_loc_description["ID"].isin(crawl_id)]

In [7]:
df_to_crawl_tags_with_link = df_id.loc[df_id["ID"].isin(df_to_crawl_tags["ID"])]

In [8]:
df_to_crawl_tags_with_link["tags"] = np.nan

C:\Users\Admin\AppData\Local\Temp\ipykernel_11732\1970817498.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_crawl_tags_with_link["tags"] = np.nan


In [9]:
df_to_crawl_tags_with_link = df_to_crawl_tags_with_link.reset_index().drop(["index"], axis=1)

In [10]:
df_to_crawl_tags_with_link

,ID,link,tags
0,112,https://www.wattpad.com/story/88620515-vampire...,NaN
1,114,https://www.wattpad.com/story/139530299-angel-...,NaN
2,554,https://www.wattpad.com/story/342694907-addictive,NaN
3,612,https://www.wattpad.com/story/298163991-good-g...,NaN
4,795,https://www.wattpad.com/story/6341473-the-bad-...,NaN
...,...,...,...
59,2767,https://www.wattpad.com/story/362849243-chim-q...,NaN
60,2788,https://www.wattpad.com/story/357238288-b%E1%B...,NaN
61,2793,https://www.wattpad.com/story/343723287-ho%C3%...,NaN
62,2794,https://www.wattpad.com/story/344552025-zhihu-...,NaN


In [11]:
def login(driver, timeout_value=10000):
    # Access the variables using os.environ.get
    USERNAME = ""
    PASSWORD = ""

    login_button = WebDriverWait(driver, timeout_value).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div[3]/div/div[2]/main/div/div/div/div/button"))
    )

    # Click the button
    login_button.click()

    username_fill = WebDriverWait(driver, timeout_value).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div[3]/div/div[2]/main/div/div/div/div/form/div/div[1]/input"))
    )

    username_fill.send_keys(USERNAME)

    password_fill = WebDriverWait(driver, timeout_value).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div[3]/div/div[2]/main/div/div/div/div/form/div/div[2]/div/input"))
    )

    password_fill.send_keys(PASSWORD)

    login_button = WebDriverWait(driver, timeout_value).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div[3]/div/div[2]/main/div/div/div/div/form/input"))
    )

    login_button.click()
    
    return driver

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import os
import time

#from dotenv import load_dotenv

#from functions import login, nav_tags, close_db, clear_db, reopen_db, stories_crawled, old_stories

###### Environment variables ######
# Load environment variables from the .env file
#load_dotenv()

###### Setup chrome driver ######
# Ignore certificate errors
options = Options()
options.add_argument('--ignore-certificate-errors')
options.add_experimental_option('excludeSwitches', ['enable-logging'])

# Set the path to your chromedriver executable
chrome_driver_path = os.path.join(os.getcwd(), "other", "chromedriver-win64", "chromedriver.exe")

# Create a new instance of the Chrome driver
driver = webdriver.Chrome(options=options)

# Open the webpage
driver.get("https://www.wattpad.com/login")

timeout_value = 10000

driver = login(driver, timeout_value)

for i in range(len(df_to_crawl_tags_with_link)):
    if pd.isna(df_to_crawl_tags_with_link.iloc[i]["tags"]) == False:
        print(i)
        continue 
        
    link = df_to_crawl_tags_with_link.iloc[i]["link"]
    index = df_to_crawl_tags_with_link.iloc[i]["ID"]
    
    driver.get(link)
    
    try:
        tags_wrapper = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "tag-items"))
        )
        
        each_tags = WebDriverWait(tags_wrapper, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "tag-pill__yrFxu"))
        )
        
        extract_tags = [each_tag.text for each_tag in each_tags]
        
        print(str(extract_tags))
        
        #break
    
        df_to_crawl_tags_with_link.at[i, "tags"] = str(extract_tags)

        if i % 5 == 4:
            print(f"Crawl to index {i}")
    except Exception as e:
        print(e)
    except:
        print(f"Error index {i}")
        id_error.append(i)

0
1
2
3
Message: 

Message: 

Message: 

Message: 

8
Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF619EDAD02+56930]
	(No symbol) [0x00007FF619E4F602]
	(No symbol) [0x00007FF619D042E5]
	(No symbol) [0x00007FF619D498ED]
	(No symbol) [0x00007FF619D49A2C]
	(No symbol) [0x00007FF619D8A967]
	(No symbol) [0x00007FF619D6BCDF]
	(No symbol) [0x00007FF619D881E2]
	(No symbol) [0x00007FF619D6BA43]
	(No symbol) [0x00007FF619D3D438]
	(No symbol) [0x00007FF619D3E4D1]
	GetHandleVerifier [0x00007FF61A256F8D+3711213]
	GetHandleVerifier [0x00007FF61A2B04CD+4077101]
	GetHandleVerifier [0x00007FF61A2A865F+4044735]
	GetHandleVerifier [0x00007FF619F79736+706710]
	(No symbol) [0x00007FF619E5B8DF]
	(No symbol) [0x00007FF619E56AC4]
	(No symbol) [0x00007FF619E56C1C]
	(No symbol) [0x00007FF619E468D4]
	BaseThreadInitThunk [0x00007FFE1DB87344+20]
	RtlUserThreadStart [0x00007FFE1F6826B1+33]

Message: stale element reference: stale element not found
  (Session info: chrome=122.0.6261.129); For documentation on t

In [18]:
len(df_to_crawl_tags_with_link.loc[df_to_crawl_tags_with_link["tags"].isnull()])

14

In [15]:
for i in range(len(df_to_crawl_tags_with_link)):
    id_story = df_to_crawl_tags_with_link.at[i, "ID"]
    tags_story = df_to_crawl_tags_with_link.at[i, "tags"]
    
    #print(id_story)
    #print(tags_story)
    
    id_df = df_loc_description.loc[df_loc_description["ID"] == id_story].index[0]
    
    df_loc_description.at[id_df, "story_tags_list"] = tags_story

In [17]:
len(df_loc_description.loc[df_loc_description["story_tags_list"].isnull()])

14

In [19]:
df_loc_description = df_loc_description.loc[df_loc_description["story_tags_list"].isnull() == False]
df_loc_description

,ID,Description,depth,story_tags_list,cover_img,title,Lượt đọc,Lượt bình chọn,Chương,recommended_stories,badges,Thời gian
0,0,Cuộc sống và chuyện tình cảm của 12 chòm sao t...,0,"['12chomsao', '12cs', '12cunghoàngđạo', 'bíẩn'...",https://img.wattpad.com/cover/51678600-512-k93...,Vampire và người sói (12 chòm sao)[full],69463.0,3355.0,32.0,"['0020', '0021', '0022', '0023', '0024', '0025...",NaN,NaN
1,1,"Thế kỷ 21, nàng là một trong những người nổi t...",0,"['bíẩn', 'cungđấu', 'dieutra', 'thriller', 'tr...",https://img.wattpad.com/cover/128268448-512-k3...,Nữ Ngỗ Tác Họa Cốt,3529078.0,128419.0,160.0,"['0030', '0031', '0032', '0033', '0034', '0035...",['Đang sáng tác'],NaN
2,2,Đây là những gì mình sưu tầm được. Vì khá thíc...,0,"['bíẩn', 'cryptic', 'fbi']",https://img.wattpad.com/cover/68364418-512-k14...,CRYPTIC ( Câu đố Cryptic Ft. Đáp án),87261.0,2780.0,13.0,"['0037', '0036', '0035', '0030', '0034', '0032...",['Đang sáng tác'],NaN
3,3,Tình yêu của một tên giết người dưới góc nhìn ...,0,"['blood', 'bíẩn', 'horror', 'killer', 'kinhdi'...",https://img.wattpad.com/cover/89250906-512-k22...,Em sẽ là nạn nhân của anh,34795.0,2049.0,17.0,"['0032', '0036', '0031', '0034', '0035', '0038...",['Hoàn thành'],NaN
4,4,Hán Việt: Tuyệt vọng giáo thất\n \n Tác ...,0,"['bíẩn', 'edit', 'hiệnđại', 'hồihộp', 'khoahọc...",https://img.wattpad.com/cover/182991023-256-k8...,LỚP HỌC RÙNG RỢN,25746.0,1068.0,NaN,"['0036', '0035', '0038', '0032', '0039', '0030...",['Đang sáng tác'],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2970,2970,ta chia tay vào một ngày lặng gió\nta tái ngộ ...,1,"['1shot', 'chill', 'oneshot', 'taeguk', 'taehy...",https://img.wattpad.com/cover/298123488-256-k6...,Phong Đỏ,1090.0,150.0,NaN,NaN,['Hoàn thành'],NaN
2971,2971,"Anh là nhà của em, mỗi khi mệt mỏi, chỉ cần nh...",1,"['chill', 'healing', 'hyunho', 'hyunjin', 'kno...",https://img.wattpad.com/cover/347124646-256-k7...,[Lee Minho x Hwang Hyunjin] Home,452.0,64.0,5.0,NaN,['Hoàn thành'],NaN
2972,2972,Maybee : OOC !!\nLệch Nguyên Tác !!!\nDo not đ...,1,"['bllk', 'bluelock', 'chill', 'kainess', 'kais...",https://img.wattpad.com/cover/356730180-256-k6...,| KAINESS | Tình cờ yêu em,81.0,4.0,1.0,NaN,['Đang sáng tác'],NaN
2973,2973,• Author:@ElinaDepChai (Wattpad)\n • Fandom...,1,"['anime', 'bias', 'charxreader', 'chill', 'fan...",https://img.wattpad.com/cover/322132697-256-k3...,• (Shikimori not Just cute/kamiya x reader/one...,NaN,NaN,NaN,NaN,['Đang sáng tác'],NaN


In [20]:
df_loc_description.to_csv("./5_crawl_tags_and_clean_null_description.csv", index=False)